<a href="https://colab.research.google.com/github/wiv33/A-Learning-python/blob/master/machine-learning/_000_hello_machine/_002_examples/submit_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

drive  sample_data


In [3]:
import numpy as np

In [4]:
import pandas as pd

df_tag_map = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/programmers/tags.csv')

In [5]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/programmers/train.csv')

In [6]:
user_tag = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/programmers/user_tags.csv')

In [7]:
job_companies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/programmers/job_companies.csv')

In [8]:
job_tag = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/programmers/job_tags.csv')

# tagID 원-핫 인코딩 base df 초기화


*   모든 tag transpose
*   train data에 concat
*   companyID merge
*   nan to 0



In [ ]:
독립 = train[:][['userID', 'jobID']]
독립

In [ ]:
tt = df_tag_map.transpose()
tt.columns = tt.values[0]
df_merge = pd.merge(독립, job_companies[:][['jobID', 'companyID']])
df_concat = pd.concat([df_merge, tt], ignore_index=True)
df_concat.drop(df_concat.index[[6000, 6001]], inplace=True)
df_independent = df_concat.fillna(0)



In [11]:
def find_column_seq(col):
  for i, x in enumerate(df_independent.columns):
    if col == x:
      return i, x

In [12]:
find_column_seq('e3251075554389fe91d17a794861d47b')

(4, 'e3251075554389fe91d17a794861d47b')

In [13]:
def update_col_in_df(update_df: pd.DataFrame, id: str, tags: []):
  for i in range(len(update_df[id])):
    if len(tags[update_df[id][i]]) > 0:
      for j in tags[update_df[id][i]]:
        j_idx = find_column_seq(j)[0]
        update_df.iloc[i: i+1, j_idx:j_idx + 1] += 1


In [14]:
# df_independent['userID'][1]
df_independent.iloc[0: 1, 3:4]

,602d1305678a8d5fdb372271e980da6a
0,0


# User Tag와 Job Tag Grouping


In [15]:
def grouping_tags(df: pd.DataFrame, id:str) -> {}:
  result = {}
  for u, t in df.groupby(id):
    result[u] = df[df[id] == u]['tagID'].values

  return result

# job_companies
## jobID 기준으로 Company 컬럼 추가,

In [ ]:
df_test = pd.merge(df_independent, job_companies[:][['jobID', 'companyID']])
df_test

# 데이터 전처리 시작

In [ ]:
# Try using .loc[row_indexer,col_indexer] = value instead

update_col_in_df(df_independent, 'userID', grouping_tags(user_tag.drop_duplicates(), 'userID'))
# df_independent
update_col_in_df(df_independent, 'jobID', grouping_tags(job_tag.drop_duplicates(), 'jobID'))

In [18]:
df_train = df_independent
df_y = train[['applied']]

In [19]:
# df_train = df_train[['userID', 'jobID', 'companyID']]

In [20]:
print(df_train.shape)
print(df_y.shape)
# from sklearn.decomposition import PCA

# pca = PCA(n_components=3)
# df_train = pca.fit_transform(df_train)
# print(df_train.shape)

(6000, 890)
(6000, 1)


# 모델 선언

In [21]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
X = tf.keras.layers.Input(shape=[890])
H = tf.keras.layers.Flatten()(X)
H = tf.keras.layers.Dropout(rate=0.2)(H)


H = tf.keras.layers.Activation(activation='swish')(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Dense(units=1000)(H)

H = tf.keras.layers.Activation(activation='swish')(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Dense(units=300)(H)

Y = tf.keras.layers.Dense(units=1, activation='sigmoid')(H)

model = tf.keras.Model(X, Y)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()
print(df_train.shape, df_y.shape)
model.fit(df_train, df_y, batch_size=128, epochs=33)

